In [57]:
###################################
#       Loading Packages          #
###################################
import cufflinks as cf
cf.go_online()

import itertools
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import glob
from scipy import stats
import seaborn as sns
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go


In [58]:
###################################
#        Imporing Data            #
###################################

#define path
path=r"/Users/valeriyamalenko/Desktop/trading/trading/momentum/data/mecklai-finance/"

#list paths to all usd-base files
files=glob.glob(path +"/usd-*.csv")
 
#open the first file
tb=pd.read_csv(files[0])

#append all other files in the directory to the first file
for i in files[1:]:
    tb1=pd.read_csv(i, header=0)
    tb=tb.append(tb1)
    
tb=tb[['Currency', 'Date', 'SpotDate', 'SpotRate', 'col1']]
tb=tb.drop_duplicates()
tb=tb.reset_index()
tb.drop('index', inplace=True, axis='columns')
tb = tb.rename(columns = {'col1': 'frd1m'})
len(tb.Currency.unique().tolist())

#list paths to usd-term files
files=glob.glob(path +"/*-usd.csv")

#open the first file
tb2=pd.read_csv(files[0])

#append all other files in the directory to the first file
for i in files[1:]:
    tb3=pd.read_csv(i, header=0)
    tb2=tb2.append(tb3)
    
tb2=tb2[['Currency', 'Date', 'SpotDate', 'SpotRate', 'col1']]
tb2=tb2.drop_duplicates()
tb2=tb2.reset_index()
tb2.drop('index', inplace=True, axis='columns')
tb2=tb2.rename(columns = {'col1': 'frd1m'})

#transorming pairs to be quoted against the dollar (e.g. USD/GBP)
tb2.SpotRate=1/tb2.SpotRate
tb2.frd1m=1/tb2.frd1m

a=tb2.Currency.str.split("/")
tb2["Currency"]=a.str[1]+"/"+a.str[0]
len(tb2.Currency.unique().tolist())

#adding usd-term and usd-base dfs
frames=[tb2,tb]
df=pd.concat(frames)
df.rename(columns = {'Date': 'date', 'Currency': 'ccy', 'SpotRate': 'spot'}, inplace = True)


##################################
#  Cleaning and Inspecting Data  #
##################################

#convert date to datetime
df.date=pd.to_datetime(df['date'])

#set date to be the index
df=df.set_index('date')

#exclude 2019 data
dt = datetime.datetime(2019, 1, 1)
df = df.iloc[df.index < dt]

df.reset_index().groupby('date').count().sort_values(by='ccy', ascending = False)

#the dataset clearly has a problem: duplicates

df= df.reset_index()  

#drop duplicates
ccy_l=df['ccy'].unique().tolist()
i=ccy_l[0]
df_1=df[df['ccy']==i]
df_1=df_1.drop_duplicates(['date'])

for i in ccy_l[1:]:
    df_2=df[df['ccy']==i]
    df_2=df_2.drop_duplicates(['date'])
    df_1=df_1.append(df_2)
df=df_1

df =df.set_index('date')

#select last day of the month for each month
df=df.loc[df.reset_index().groupby(df.index.to_period('M'))['date'].max()]
len(df) # good!  12*24*8 = 2304

df.reset_index(inplace = True)

In [59]:
##################################
#      Transofrming Data         #
##################################

#below is included to suppress the warning
pd.options.mode.chained_assignment = None

#1#generate new cols for logs of spot and fwd rates
df['logSpotRate']=np.log(df['spot'])
df['logfwd1m']=np.log(df['frd1m'])

#2#shift forward rates 
ret_df = df[['ccy', 'logSpotRate', 'logfwd1m', 'date']]
ret_df = ret_df.pivot_table(index = 'date', columns = ['ccy'])

for i in df.ccy.unique().tolist():
    ret_df[('logfwd1m', i)] =  ret_df[('logfwd1m', i)].shift(1)
ret_df = ret_df[1:]

for i in df.ccy.unique().tolist():
    ret_df[('ret', i)] = ret_df[('logfwd1m', i)] - ret_df[('logSpotRate', i)] 

ret_df = pd.DataFrame(ret_df['ret'].unstack())
ret_df.reset_index(inplace=True)
ret_df.set_index('date', inplace = True)
ret_df.reset_index(inplace=True)
ret_df.rename(columns = {0: 'spot'}, inplace = True)

In [60]:
##################################
# Create Momentum Portfolio      #
##################################

N = 3 # number of portfolios
J = [1, 3, 6, 9, 12] # formation Period Length: J can be between 3 to 12 months
K =[1, 3, 6, 9, 12] # folding Period Length: K can be between 3 to 12 months

ret_df = ret_df[['ccy','date','spot']].sort_values(['ccy','date']).set_index('date')

#falculate rolling cumulative return
#by summing log(1+ret) over the formation period
ret_df['logret']=np.log(1+ret_df['spot'])

In [61]:
df_frames = {}
df_cumulative = {}
for j,k in itertools.product(J, K):
    ret_df1 = ret_df.copy()
    umd = ret_df1.groupby(['ccy'])['logret'].rolling(j, min_periods=j).sum()
    umd = umd.reset_index()
    umd['cumret']=np.exp(umd['logret'])-1
    
    
     ##################################
     #   Formation of 3  Portfolios   #
     ##################################
    
    #for each date: assign ranking 1-3 based on cumret 1=lowest 3=highest cumret
    umd=umd.dropna(axis=0, subset=['cumret'])
    umd['momr']=umd.groupby('date')['cumret'].transform(lambda x: pd.qcut(x, N, labels=False))
    umd.momr=umd.momr.astype(int)
    umd['momr'] = umd['momr']+1
    umd['hdate1']=umd['date']+pd.tseries.offsets.MonthBegin(1)
    umd['hdate2']=umd['date']+pd.tseries.offsets.MonthEnd(k)
    umd=umd.rename(columns={'date':'form_date'})
    umd = umd[['ccy','form_date','momr','hdate1','hdate2']]
    
    #join rank and return data together
    #note: this step consumes a lot of memory so takes a while
    ret_df1.reset_index(inplace = True)
    _tmp_ret = ret_df1[['ccy','date','spot']]
    port = pd.merge(_tmp_ret, umd, on=['ccy'], how='inner')
    port = port[(port['hdate1']<=port['date']) & (port['date']<=port['hdate2'])]
    
    umd2 = port.sort_values(by=['date','momr','form_date','ccy']).drop_duplicates()
    umd3 = umd2.groupby(['date','momr','form_date'])['spot'].mean().reset_index()
    
    #skip first two years of the sample 
    start_yr = umd3['date'].dt.year.min()+2
    umd3 = umd3[umd3['date'].dt.year>=start_yr]
    umd3 = umd3.sort_values(by=['date','momr'])
    
    #create one return series per MOM group every month
    ewret = umd3.groupby(['date','momr'])['spot'].mean().reset_index()
    ewstd = umd3.groupby(['date','momr'])['spot'].std().reset_index()
    ewret = ewret.rename(columns={'spot':'ewret'})
    ewstd = ewstd.rename(columns={'spot':'ewretstd'})
    ewretdat = pd.merge(ewret, ewstd, on=['date','momr'], how='inner')
    ewretdat = ewretdat.sort_values(by=['momr', 'date'])
    ewretdat
    # portfolio summary
    ewretdat.groupby(['momr'])['ewret'].describe()[['count', 'mean', 'std']]
    
    
    ##################################
    #  Long-Short Portfolio Returns  #
    ##################################
    
    #transpose portfolio layout to have columns as portfolio returns
    ewretdat2 = ewretdat.pivot(index='date', columns='momr', values='ewret')
    
    #add prefix port in front of each column
    ewretdat2 = ewretdat2.add_prefix('port')
    ewretdat2 = ewretdat2.rename(columns={'port1':'losers', 'port'+str(N):'winners'})
    ewretdat2['long_short'] = ewretdat2['winners'] - ewretdat2['losers']
    
    #annualised return
    ewretdat3=ewretdat2[['long_short']].add(1).prod() ** (12 / len(ewretdat2['long_short'])) - 1
    ewretdat3 = ewretdat3.to_frame().rename(columns={0:'Excess Return'}).reset_index()
    
    ewretdat3['f'] = j
    ewretdat3['h'] = k
    ewretdat3[['T-Statistic','p-value']] = pd.Series(stats.ttest_1samp(ewretdat2['long_short'],0.0)).to_frame().T
    ewretdat3['Sharpe Ratio'] = (ewretdat2['long_short'].mean()*12)/(ewretdat2['long_short'].std()*np.sqrt(12))
    df_frames[(j,k)] = ewretdat3
   
    #compute Long-Short Portfolio Cumulative Returns
    ewretdat4 = ewretdat2
    ewretdat4['1+losers']=1+ewretdat2['losers']
    ewretdat4['1+winners']=1+ewretdat2['winners']
    ewretdat4['1+ls'] = 1+ewretdat2['long_short']
    
    ewretdat4['cumret_winners']=ewretdat4['1+winners'].cumprod()-1
    ewretdat4['cumret_losers']=ewretdat4['1+losers'].cumprod()-1
    ewretdat4['cumret_long_short']=ewretdat4['1+ls'].cumprod()-1
    ewretdat4['f'] = j
    ewretdat4['h'] = k
    df_cumulative[(j,k)]=ewretdat4

df = pd.concat(df_frames.values(), ignore_index=True)
df = df.pivot_table(values =['T-Statistic','Excess Return', 'Sharpe Ratio'], 
                    columns ='h', index = 'f')

df1 = df.style.format({
    ("Excess Return", 1): lambda x: "{:.2%}".format(x),
    ("Excess Return", 3): lambda x: "{:.2%}".format(x),
    ("Excess Return", 6): lambda x: "{:.2%}".format(x),
    ("Excess Return", 9): lambda x: "{:.2%}".format(x),
    ("Excess Return", 12): lambda x: "{:.2%}".format(x),
    
    ("Sharpe Ratio", 1): lambda x: "{:.2}".format(x),
    ("Sharpe Ratio", 3): lambda x: "{:.2}".format(x),
    ("Sharpe Ratio", 6): lambda x: "{:.2}".format(x),
    ("Sharpe Ratio", 9): lambda x: "{:.2}".format(x),
    ("Sharpe Ratio", 12): lambda x: "{:.2}".format(x),
    
    ("T-Statistic", 1): lambda x: "{:.2}".format(x),
    ("T-Statistic", 3): lambda x: "{:.2}".format(x),
    ("T-Statistic", 6): lambda x: "{:.2}".format(x),
    ("T-Statistic", 9): lambda x: "{:.2}".format(x),
    ("T-Statistic", 12): lambda x: "{:.2}".format(x)}).set_caption('Table 1. Momentum Returns and Sharpe Ratios')

In [62]:
cm = sns.light_palette("green", as_cmap=True);
df1.background_gradient(cmap=cm, low =0, high=0.99);

In [63]:
df = pd.concat(df_cumulative.values(), ignore_index=False)
df['f/h'] = list(zip(df.f, df.h))
df.drop(['h', 'f'], axis=1, inplace=True)
df.reset_index(inplace=True)

In [64]:
###############################################
#           Visualising the data              #
##############################################

#ret_df.pivot(columns = 'ccy', index = 'date', values = 'spot')\
#.plot(kind='line', subplots=True, grid=True, title="Currency Monthly Returns",
#         sharex=True, sharey=True, legend=True, figsize=(16, 16), layout=(6,4),
#       style=['g', 'b', 'b', 'b', 'r', 'r', 'b', 'b', 'g','r', 'r', 'b','r', 'b', 'b','r', 'g', 'r','r', 'r', 'r']);

In [65]:
dataPanda = [];
for i in df['f/h'].unique().tolist():
    new =df[df['f/h']==i]
    trace = go.Scatter(
    x = list(new['date']),
    y = list(new['cumret_long_short']*100), 
    name=str(i))
    
    dataPanda.append(trace) 
layout = go.Layout(
    title='Figure 1. Cumulative Excess returns of momentum strategies',
    yaxis=dict(
        title='Cumulative Excess Returns (in %)',
        titlefont=dict(
            size=18,
            color='#7f7f7f')),
    width=1000,
    height=700,
 annotations=[
        dict(
            x=1.09,
            y=1.03,
            align="center",
            valign="top",
            text='f, h',
            showarrow=False,
            xref="paper",
            yref="paper",
            xanchor="center",
            yanchor="top")])
     
fig = dict(data=dataPanda,layout=layout )
config={'showLink': False} 
#iplot(fig, config=config)